# YAML To CFG

### 注意事項: 所有模塊都能修改 除了 route_position_chenk_translate 與 yaml_Route_cfg_translate 兩個函式
### 這兩塊函式動了程式就會爆炸 !!! 爆炸!!!!!
### 只要有重新指向功能的函式就會用到 route_assign_list跟 layer_list 這兩個變數(實際上

## 一. 程式讀檔

In [1]:
import yaml
import numpy as np
import copy
import math

# 程式讀檔
file_name = 'yolov7'
with open(file_name+'.yaml', 'r' ,encoding="utf-8") as f:
    # 這裡會將 yaml的檔案拆成好讀的樣子 想知道裡面長怎樣的話直接 print(data) print(data['backbone'])看看就知道
    data = yaml.safe_load(f)

## 二. 函式定義 將底層功能與大型模塊實現

### 重要參數說明:

global route_count : 紀錄cfg當下層數，這個會在yaml每一層轉換完成後存入layer_list，這樣就會有每一層yaml對應的cfg層數。

layer_list : 在每一層layer轉換完成後記錄當下的層數，這個list會在[route]指向的時候映射到正確的位置。

route_assign_list : 把有指向到別層動作的[conv]層記錄下來，這個list是給你比對檢查用的，看多層指向的時候有沒有指錯。

In [2]:
# 神經網路超參數定義 就是懶得自己再貼參數啦 如果要改成自定義也不是不行
def yolo_param(cfg_file):
    cfg_file.writelines("[net]"+"\n")
    cfg_file.writelines("subdivisions = "+str(32)+"\n")
    cfg_file.writelines("width = "+str(640)+"\n")
    cfg_file.writelines("height = "+str(640)+"\n")
    cfg_file.writelines("channels = "+str(3)+"\n")
    cfg_file.writelines("momentum = "+str(0.949)+"\n")
    cfg_file.writelines("decay = "+str(0.0005)+"\n")
    cfg_file.writelines("angle = "+str(0)+"\n")
    cfg_file.writelines("saturation = "+str(1)+"\n")
    cfg_file.writelines("exposure = "+str(1.5)+"\n")
    cfg_file.writelines("hue = "+str(.3)+"\n")
    
    cfg_file.writelines("learning_rate = "+str(0.01)+"\n")
    cfg_file.writelines("burn_in = "+str(1000)+"\n")
    cfg_file.writelines("max_batches = "+str(4000)+"\n")
    cfg_file.writelines("policy = "+str('steps')+"\n")
    cfg_file.writelines("steps = "+str('2000,3000')+"\n")
    cfg_file.writelines("scales = "+str('.1,.1')+"\n")
    
    cfg_file.writelines("\n")
    cfg_file.writelines("\n")

In [3]:
# 這邊是這隻程式的核心算法 很簡單 也很複雜 嘔心瀝血之作 XD
# 這隻程式在做的事情是判斷是多層route指向或單層指向，然後做轉換

def route_position_chenk_translate(cfg_file, cfg_layers, route_assign_list, layer_list):
    # 這個是在數現在在cfg中到了第幾層 做法是在每一層layer轉換後+1 並記錄到layer_list 這樣就會有每一層layer對應的實際層數
    global route_count
    
    # 判斷需要指向的層數是單一層或多層 這邊是多層的狀況
    if len(cfg_layers)>1:
        # 將多層layer中每一層做轉換，變成cfg實際層數。
        for i in range(len(cfg_layers)):
            cfg_layers[i] = layer_list[cfg_layers[i]]
        # 這裡print可以檢查conv指向的位置跟concat指向的位置有沒有正確
        print("cfg_layers:",cfg_layers)
        # 丟給可愛的route函式
        yaml_Route_cfg_translate(cfg_file, cfg_layers, route_assign_list, layer_list)
        
    # 單層的狀況   
    else:
        # 如果conv指向是指到-1就不做處理，因為預設就是-1，可以直接處理
        if cfg_layers[0] != -1:
            # 將layer做轉換成cfg實際層數。
            cfg_layers[0] = layer_list[cfg_layers[0]]
            # 因為單層的狀況會導致cfg層數+1，所以route會受影響，這邊是將有單層指向的層數記錄下來，之後可以用來做檢查。
            route_assign_list.append(layer_list[-1])
            # 丟給可愛的route函式
            yaml_Route_cfg_translate(cfg_file, cfg_layers, route_assign_list, layer_list)

In [4]:
# 這層就很簡單的做寫入的動作 ･◡･

def yaml_Route_cfg_translate(cfg_file, layers, route_assign_list, layer_list): 
    
    global route_count
    cfg_file.writelines("## --------------------------------------"+"yaml:"+str(len(layer_list))+"\tcfg:"+str(route_count+1)+"--------------------------------------\n\n")
    # 這邊就很簡單的將route寫入cfg 然後這邊可能會有多層layer 所以使用偷懶的寫法可套用所有狀況 如果格式正確的話。
    cfg_file.writelines("[route]"+"\n")
    cfg_file.writelines("layers = "+','.join(str(layer) for layer in layers)+"\n")
    cfg_file.writelines("\n")

    route_count = route_count + 1

In [5]:
# 神經網路的核心元件 也很簡單 只有放一層route跟寫入的動作
def yaml_conv_cfg_translate(cfg_file, route_assign_list, layer_list, cfg_data, batch_normalize, padding, activation):

    global route_count

    # 將filter值參數拆出來，讓下面可以直接使用
    cfg_parm = cfg_data[3]
    # 這邊將route放在 [conv]之前，因為如果是指向前面的層數的話，是將指向層數的屁股接到自己的頭，所以放前面
    route_position_chenk_translate(cfg_file, [cfg_data[0]], route_assign_list, layer_list)
    
    cfg_file.writelines("## --------------------------------------"+"yaml:"+str(len(layer_list))+"\tcfg:"+str(route_count+1)+"--------------------------------------\n\n")

    # 快樂的寫入･◡･
    cfg_file.writelines("[convolutional]"+"\n")
    cfg_file.writelines("batch_normalize = "+str(batch_normalize)+"\n")
    cfg_file.writelines("filters = "+str(cfg_parm[0])+"\n")
    cfg_file.writelines("size = "+str(cfg_parm[1])+"\n")
    cfg_file.writelines("stride = "+str(cfg_parm[2])+"\n")
    cfg_file.writelines("pad = "+str(padding)+"\n")
    cfg_file.writelines("activation = "+str(activation)+"\n")
    cfg_file.writelines("\n")
    
    route_count = route_count + 1
    print(route_count)
    

In [6]:
def yaml_SP_cfg_translate(cfg_file, route_assign_list, layer_list, cfg_data, size):
    global route_count

    cfg_parm = cfg_data[3]
    
    # 與conv同理 這邊也會做指向的動作
    route_position_chenk_translate(cfg_file, [cfg_data[0]], route_assign_list, layer_list)
    
    cfg_file.writelines("## --------------------------------------"+"yaml:"+str(len(layer_list))+"\tcfg:"+str(route_count+1)+"--------------------------------------\n\n")

    # 寫入 ･◡･ 不過這邊要注意 這裡的 pad是 size/2 common中是這樣寫的 如果不高興也可以改
    cfg_file.writelines("[maxpool]"+"\n")
    cfg_file.writelines("size = "+str(cfg_parm[0])+"\n")
    cfg_file.writelines("stride = "+str(1)+"\n")
    cfg_file.writelines("pad = "+str(int(round(cfg_parm[0]/2)))+"\n")
    cfg_file.writelines("\n")
    
    route_count = route_count + 1
    print(route_count)

In [7]:
def yaml_Max_Pooling_cfg_translate(cfg_file, route_assign_list, layer_list, cfg_data, stride, size):    
    
    global route_count
    
    route_position_chenk_translate(cfg_file, [cfg_data[0]], route_assign_list, layer_list)

    
    cfg_file.writelines("## --------------------------------------"+"yaml:"+str(len(layer_list))+"\tcfg:"+str(route_count+1)+"--------------------------------------\n\n")
    print('maxpool')
    # maxpooling寫入 ･◡･
    cfg_file.writelines("[maxpool]"+"\n")
    cfg_file.writelines("stride = "+str(stride)+"\n")
    cfg_file.writelines("size = "+str(size)+"\n")
    cfg_file.writelines("\n")
    
    route_count = route_count + 1
    print(route_count)

In [8]:
def yaml_upsample_cfg_translate(cfg_file, route_assign_list, layer_list, cfg_data, stride):
    
    global route_count
    
    route_position_chenk_translate(cfg_file, [cfg_data[0]], route_assign_list, layer_list)
    
    cfg_file.writelines("## --------------------------------------"+"yaml:"+str(len(layer_list))+"\tcfg:"+str(route_count+1)+"--------------------------------------\n\n")

    # upsample 寫入･◡･
    cfg_file.writelines("[upsample]"+"\n")
    cfg_file.writelines("stride = "+str(stride)+"\n")
    cfg_file.writelines("\n")
    
    route_count = route_count + 1
    print(route_count)

In [9]:
# SPPCSPC寫入 這邊是直接將 SPPCSPC照翻
def yaml_SPPCSPC_cfg_translate(cfg_file, route_assign_list, filter_num, activation, batch_normalize, padding):  

    global route_count
    
    cfg_file.writelines("\n## SPPCSPC start\n\n")

    yaml_conv_cfg_translate(cfg_file, route_assign_list, layer_list,[-1, 1, 'Conv', [filter_num, 1, 1]],batch_normalize, padding, activation)
    yaml_conv_cfg_translate(cfg_file, route_assign_list, layer_list,[-2, 1, 'Conv', [512, 1, 1]],batch_normalize, padding, activation)
    yaml_conv_cfg_translate(cfg_file, route_assign_list, layer_list,[-1, 1, 'Conv', [512, 3, 1]],batch_normalize, padding, activation)
    yaml_conv_cfg_translate(cfg_file, route_assign_list, layer_list,[-1, 1, 'Conv', [512, 1, 1]],batch_normalize, padding, activation)
    
    yaml_Route_cfg_translate(cfg_file, [-1], route_assign_list, layer_list)
    yaml_Max_Pooling_cfg_translate(cfg_file, route_assign_list, layer_list, [-1], 1, 5)
    yaml_Route_cfg_translate(cfg_file, [-3], route_assign_list, layer_list)
    yaml_Max_Pooling_cfg_translate(cfg_file, route_assign_list, layer_list, [-1], 1, 9)
    yaml_Route_cfg_translate(cfg_file, [-5], route_assign_list, layer_list)
    yaml_Max_Pooling_cfg_translate(cfg_file, route_assign_list, layer_list, [-1], 1, 13)
    
    
    
    yaml_Route_cfg_translate(cfg_file, [-1,-3,-5,-7], route_assign_list, layer_list)
    
    yaml_conv_cfg_translate(cfg_file, route_assign_list, layer_list,[-1, 1, 'Conv', [512, 1, 1]],batch_normalize, padding, activation)
    yaml_conv_cfg_translate(cfg_file, route_assign_list, layer_list,[-1, 1, 'Conv', [512, 3, 1]],batch_normalize, padding, activation)

    yaml_Route_cfg_translate(cfg_file, [-1,-14], route_assign_list, layer_list)
    yaml_conv_cfg_translate(cfg_file, route_assign_list, layer_list,[-1, 1, 'Conv', [512, 1, 1]],batch_normalize, padding, activation)

    cfg_file.writelines("## SPPCSPC end\n\n")

In [10]:
# shortcut寫入 這個元件在圖中是 add的原件
def  yaml_Shorcut_cfg_translate(cfg_file, add_layer_from, activation):
    
    global route_count
    cfg_file.writelines("## --------------------------------------"+"yaml:"+str(len(layer_list))+"\tcfg:"+str(route_count+1)+"--------------------------------------\n\n")

    cfg_file.writelines("[shortcut]"+"\n")
    # 這邊一樣有會指向多層的動作，所以依樣畫葫蘆
    cfg_file.writelines("from = "+','.join(str(layer) for layer in add_layer_from)+"\n")
    cfg_file.writelines("activation = "+str(activation)+"\n")
    cfg_file.writelines("\n")
    
    route_count = route_count + 1
    print(route_count)

In [11]:
# 這邊是將 REPConv實現 直接照寫出來 
def yaml_RepConv_cfg_translate(cfg_file, route_assign_list, layer_list, cfg_data, batch_normalize, padding, activation):  
    global route_count
    cfg_file.writelines("\n## RepConv start\n\n")
    
    # route
    route_position_chenk_translate(cfg_file, [cfg_data[0]], route_assign_list, layer_list)

    cfg_parm = cfg_data[3]
    cfg_file.writelines("## --------------------------------------"+"yaml:"+str(len(layer_list))+"\tcfg:"+str(route_count+1)+"--------------------------------------\n\n")

    # rep 第一層
    cfg_file.writelines("[convolutional]"+"\n")
    cfg_file.writelines("batch_normalize = "+str(batch_normalize)+"\n")
    cfg_file.writelines("filters = "+str(int(cfg_parm[0]/2))+"\n")
    cfg_file.writelines("\n")
    
    # route 啊這邊記得+1 因為這不是用函式的conv不會自動+1 所以這裡要手動+1 下面同理
    route_count = route_count + 1
    route_position_chenk_translate(cfg_file, [cfg_data[0]], route_assign_list, layer_list)
    
    cfg_file.writelines("## --------------------------------------"+"yaml:"+str(len(layer_list))+"\tcfg:"+str(route_count+1)+"--------------------------------------\n\n")

    cfg_file.writelines("[convolutional]"+"\n")
    cfg_file.writelines("batch_normalize = "+str(batch_normalize)+"\n")
    cfg_file.writelines("filters = "+str(cfg_parm[0])+"\n")
    cfg_file.writelines("size = "+str(cfg_parm[1])+"\n")
    cfg_file.writelines("stride = "+str(cfg_parm[2])+"\n")
    cfg_file.writelines("pad = "+str(padding)+"\n")
    cfg_file.writelines("\n")
    
    route_count = route_count + 1
    route_position_chenk_translate(cfg_file, [cfg_data[0]], route_assign_list, layer_list)
    
    cfg_file.writelines("## --------------------------------------"+"yaml:"+str(len(layer_list))+"\tcfg:"+str(route_count+1)+"--------------------------------------\n\n")

    cfg_file.writelines("[convolutional]"+"\n")
    cfg_file.writelines("batch_normalize = "+str(batch_normalize)+"\n")
    cfg_file.writelines("filters = "+str(cfg_parm[0])+"\n")
    cfg_file.writelines("size = "+str(cfg_parm[1])+"\n")
    cfg_file.writelines("stride = "+str(cfg_parm[2])+"\n")
    cfg_file.writelines("pad = "+str(padding)+"\n")
    cfg_file.writelines("\n")
    
    route_count = route_count + 1
    # 這邊再接一個 shortcut 然後層數是固定-3 and -5所以這邊寫死
    yaml_Shorcut_cfg_translate(cfg_file, [-3,-5], activation)
    
    # REP最後都直接接輸出 直接來吧
#     cfg_file.writelines("## --------------------------------------"+"yaml:"+str(len(layer_list))+"\tcfg:"+str(route_count+1)+"--------------------------------------\n\n")
#     cfg_file.writelines("[yolo]"+"\n")
    
#     route_count = route_count + 1
    print(route_count)

    cfg_file.writelines("\n## RepConv end\n\n")

In [12]:
# 這邊是 REP專用的尾巴 但好像不需要用到 我直接寫在REP裡面了 哈
def yaml_Yolo_cfg_translate(cfg_file, route_assign_list, layer_list, cfg_data):  

    position = cfg_data[0]
    
    for i in position:
        
        route_position_chenk_translate(cfg_file, [i], route_assign_list, layer_list)
        
        cfg_file.writelines("## --------------------------------------"+"yaml:"+str(len(layer_list))+"\tcfg:"+str(route_count+1)+"--------------------------------------\n\n")

        cfg_file.writelines("[yolo]"+"\n")
        cfg_file.writelines("\n")
    
    
    cfg_file.writelines("\n")

## 三. 程式主體 讀取Yaml並使用自定義函數轉寫成cfg

In [13]:
yaml_write_data = []
# 這邊可以自己修改
activation = 'leakyrelu'
batch_normalize = 1
padding = 1

layer_list = []

# 初始化 route位置計數
route_count = -1
# 初始化 conv指項紀錄列表
route_assign_list = []
# 這邊開啟要寫入的檔案 這裡偷懶直接把檔名用變數存起來 這樣我只要改一次
with open (file_name+'.cfg','w') as cfg_file:
    # 先把超參數都寫進來
    yolo_param(cfg_file)
    # 用 for迴圈
    for all_component in data:
        print('----------------------'+all_component+'----------------------')
        # 這邊就是把 cfg格式寫好 遇到 backbone跟 head就寫出來 然後再往下
        if all_component == 'backbone':
            yaml_write_data = data['backbone']
            cfg_file.writelines("##Backbone:\n")
        elif all_component == 'head':
            yaml_write_data = data['head']
            cfg_file.writelines("##Head:\n")
            
        # 當進入backbone跟 head的時候判斷對應的每一層要接到甚麼函式，然後直接寫入
        for i in yaml_write_data:
            
            if i[2] == 'Conv':
                print('Conv')
                yaml_conv_cfg_translate(cfg_file, route_assign_list, layer_list, i, batch_normalize, padding, activation)

            elif i[2] == 'SPPCSPC':
                print('SPPCSPC')
                filter_num = i[-1][0]
                yaml_SPPCSPC_cfg_translate(cfg_file, route_assign_list, filter_num, activation, batch_normalize, padding)

            elif i[2] == 'Concat':
                print('Concat')
                route_position_chenk_translate(cfg_file, i[0], route_assign_list, layer_list)

            elif i[2] == 'MP':
                print('MP')
                yaml_Max_Pooling_cfg_translate(cfg_file, route_assign_list, layer_list, i, 2, 2)

            elif i[2] == 'nn.Upsample':
                print('nn.Upsample')
                yaml_upsample_cfg_translate(cfg_file, route_assign_list, layer_list, i, 2)

            elif i[2] == 'RepConv':
                print('RepConv')
                yaml_RepConv_cfg_translate(cfg_file, route_assign_list, layer_list, i, batch_normalize, padding, 'silu')

            elif i[2] == 'SP':
                print('SP')
                yaml_SP_cfg_translate(cfg_file, route_assign_list, layer_list, i, 1)

            elif i[2] == 'Detect':
                print('Detect')
                yaml_Yolo_cfg_translate(cfg_file, route_assign_list, layer_list, i)

            layer_list.append(route_count)
                

----------------------nc----------------------
----------------------depth_multiple----------------------
----------------------width_multiple----------------------
----------------------anchors----------------------
----------------------backbone----------------------
Conv
0
Conv
1
Conv
2
Conv
3
Conv
4
Conv
6
Conv
7
Conv
8
Conv
9
Conv
10
Concat
cfg_layers: [10, 8, 6, 4]
Conv
12
MP
maxpool
13
Conv
14
Conv
16
Conv
17
Concat
cfg_layers: [17, 14]
Conv
19
Conv
21
Conv
22
Conv
23
Conv
24
Conv
25
Concat
cfg_layers: [25, 23, 21, 19]
Conv
27
MP
maxpool
28
Conv
29
Conv
31
Conv
32
Concat
cfg_layers: [32, 29]
Conv
34
Conv
36
Conv
37
Conv
38
Conv
39
Conv
40
Concat
cfg_layers: [40, 38, 36, 34]
Conv
42
MP
maxpool
43
Conv
44
Conv
46
Conv
47
Concat
cfg_layers: [47, 44]
Conv
49
Conv
51
Conv
52
Conv
53
Conv
54
Conv
55
Concat
cfg_layers: [55, 53, 51, 49]
Conv
57
----------------------head----------------------
SPPCSPC
58
60
61
62
maxpool
64
maxpool
66
maxpool
68
70
71
73
Conv
74
nn.Upsample
75
Conv
77
Co

In [14]:
print("這是有單層指向狀況的layer，紀錄檢查用")
print(route_assign_list)

這是有單層指向狀況的layer，紀錄檢查用
[4, 14, 19, 29, 34, 44, 49, 57, 75, 79, 89, 93, 103, 108, 118, 123, 131, 131, 131, 138, 138, 138, 145, 145, 145, 152, 152, 152]
